## 딥러닝 2주차

### 인공신경망(ANN)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [2]:
# 데이터셋 전처리
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# MNIST 데이터셋 로드
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

In [ ]:
import matplotlib.pyplot as plt

img, label = next(iter(trainloader))

plt.imshow(img[0, 0, :, :], 'gray')
plt.title(f'{label[0]}')
plt.show()


In [4]:
class SimpleANN(nn.Module):
    def __init__(self):
        super(SimpleANN, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)  # 입력층에서 은닉층으로
        self.fc2 = nn.Linear(128, 64)       # 은닉층에서 은닉층으로
        self.fc3 = nn.Linear(64, 10)        # 은닉층에서 출력층으로

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # 입력 이미지를 1차원 벡터로 변환
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [5]:
# 모델 초기화
model = SimpleANN()

# 손실 함수와 최적화 알고리즘 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# 모델 학습
for epoch in range(10):  # 10 에포크 동안 학습
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        # 기울기 초기화
        optimizer.zero_grad()

        # 순전파 + 역전파 + 최적화
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # 손실 출력
        running_loss += loss.item()
        if i % 100 == 99:  # 매 100 미니배치마다 출력
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Finished Training')

[Epoch 1, Batch 100] loss: 1.369
[Epoch 1, Batch 200] loss: 0.502
[Epoch 1, Batch 300] loss: 0.378
[Epoch 1, Batch 400] loss: 0.348
[Epoch 1, Batch 500] loss: 0.296
[Epoch 1, Batch 600] loss: 0.276
[Epoch 1, Batch 700] loss: 0.264
[Epoch 1, Batch 800] loss: 0.247
[Epoch 1, Batch 900] loss: 0.231
[Epoch 2, Batch 100] loss: 0.198
[Epoch 2, Batch 200] loss: 0.208
[Epoch 2, Batch 300] loss: 0.189
[Epoch 2, Batch 400] loss: 0.183
[Epoch 2, Batch 500] loss: 0.180
[Epoch 2, Batch 600] loss: 0.176
[Epoch 2, Batch 700] loss: 0.180
[Epoch 2, Batch 800] loss: 0.158
[Epoch 2, Batch 900] loss: 0.148
[Epoch 3, Batch 100] loss: 0.132
[Epoch 3, Batch 200] loss: 0.146
[Epoch 3, Batch 300] loss: 0.141
[Epoch 3, Batch 400] loss: 0.132
[Epoch 3, Batch 500] loss: 0.134
[Epoch 3, Batch 600] loss: 0.124
[Epoch 3, Batch 700] loss: 0.127
[Epoch 3, Batch 800] loss: 0.120
[Epoch 3, Batch 900] loss: 0.131
[Epoch 4, Batch 100] loss: 0.112
[Epoch 4, Batch 200] loss: 0.108
[Epoch 4, Batch 300] loss: 0.119
[Epoch 4, 

In [6]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')


Accuracy of the network on the 10000 test images: 97.71%


### 합성곱 신경망(CNN)

In [7]:
# 데이터셋 전처리
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# CIFAR-10 데이터셋 로드
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# CIFAR-10의 클래스 이름
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

img, label = next(iter(trainloader))

# 첫 번째 이미지의 채널 순서를 (채널, 높이, 너비) -> (높이, 너비, 채널)로 변환
img = np.transpose(img[0], (1, 2, 0))  # (32, 32, 3)

plt.imshow(img)
plt.title(f'{classes[label[0]]}')
plt.show()

In [9]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)  # 입력 채널 3, 출력 채널 32, 커널 크기 3x3
        self.pool = nn.MaxPool2d(2, 2)               # 풀링 크기 2x2
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) # 입력 채널 32, 출력 채널 64, 커널 크기 3x3
        self.fc1 = nn.Linear(64 * 8 * 8, 512)        # 완전 연결 층
        self.fc2 = nn.Linear(512, 10)                # 출력 층 (10개의 클래스)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)  # 플래튼
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [10]:
# 모델 초기화
model_cnn = SimpleCNN()

# 손실 함수와 최적화 알고리즘 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_cnn.parameters(), lr=0.01, momentum=0.9)

# 모델 학습
for epoch in range(10):  # 10 에포크 동안 학습
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        # 기울기 초기화
        optimizer.zero_grad()

        # 순전파 + 역전파 + 최적화
        outputs = model_cnn(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # 손실 출력
        running_loss += loss.item()
        if i % 100 == 99:  # 매 100 미니배치마다 출력
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Finished Training')

[Epoch 1, Batch 100] loss: 2.108
[Epoch 1, Batch 200] loss: 1.773
[Epoch 1, Batch 300] loss: 1.581
[Epoch 1, Batch 400] loss: 1.478
[Epoch 1, Batch 500] loss: 1.413
[Epoch 1, Batch 600] loss: 1.337
[Epoch 1, Batch 700] loss: 1.279
[Epoch 2, Batch 100] loss: 1.153
[Epoch 2, Batch 200] loss: 1.137
[Epoch 2, Batch 300] loss: 1.099
[Epoch 2, Batch 400] loss: 1.075
[Epoch 2, Batch 500] loss: 1.045
[Epoch 2, Batch 600] loss: 1.037
[Epoch 2, Batch 700] loss: 0.998
[Epoch 3, Batch 100] loss: 0.885
[Epoch 3, Batch 200] loss: 0.886
[Epoch 3, Batch 300] loss: 0.876
[Epoch 3, Batch 400] loss: 0.885
[Epoch 3, Batch 500] loss: 0.857
[Epoch 3, Batch 600] loss: 0.847
[Epoch 3, Batch 700] loss: 0.837
[Epoch 4, Batch 100] loss: 0.735
[Epoch 4, Batch 200] loss: 0.715
[Epoch 4, Batch 300] loss: 0.718
[Epoch 4, Batch 400] loss: 0.700
[Epoch 4, Batch 500] loss: 0.687
[Epoch 4, Batch 600] loss: 0.699
[Epoch 4, Batch 700] loss: 0.708
[Epoch 5, Batch 100] loss: 0.565
[Epoch 5, Batch 200] loss: 0.556
[Epoch 5, 

In [11]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model_cnn(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')


Accuracy of the network on the 10000 test images: 73.27%
